In [ ]:
#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}!kill -9 -1MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=75171e46f09149844eb5c8cebaa8ee4579125eb93330cfd2647b671a1fe3a204
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil
Gen RAM Free: 11.7 GB  | Proc size: 1.4 GB
GPU RAM Free: 14792MB | Used: 309MB | Util   2% | Total 15360!kill -9 -1MB


In [ ]:
import datetime
from google.colab import drive
from time import time
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymannkendall
import pymannkendall as mk

In [ ]:
filtered_works = pd.read_csv("/content/drive/MyDrive/NREGAproject/newestfilteredworks.csv", index_col=0, parse_dates=True)
# filtered_works = filtered_works.drop(['workcode', 'work_name', 'sno.', '_merge', 'index', 'work_status', 'bhu_creationtime', 'bhu_serial_no', 'bhu_collection_sno', 'sanction_amount_in(lakh.)'
# ,'total_amount_paid_since_inception_in(lakh.)', 'master_work_category_name', 'total_mandays', 'no._of_units', 'is_secure', 'is_convergence', 'bhu_accuracy', 'bhu_observername'
# ,'bhu_path1', 'bhu_path2', 'agency_name'], axis=1)
filtered_works['district_name'] = filtered_works['district_name'].str.lower()
filtered_works['block_name'] = filtered_works['block_name'].str.lower()
filtered_works['map_gpname'] = filtered_works['map_gpname'].str.lower()
filtered_works['work_physically/_completed_date'] = pd.to_datetime(filtered_works['work_physically/_completed_date'])
filtered_works['work_started_date'] = pd.to_datetime(filtered_works['work_started_date'])
listofgps= filtered_works['map_gpname'].unique()

<ipython-input-5-e313472d227a>:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  filtered_works['work_physically/_completed_date'] = pd.to_datetime(filtered_works['work_physically/_completed_date'])
<ipython-input-5-e313472d227a>:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  filtered_works['work_started_date'] = pd.to_datetime(filtered_works['work_started_date'])


In [ ]:
listofblocks = filtered_works['district_name'].unique()

In [ ]:
listofblocks

array(['chikkaballapura', 'bengaluru rural', 'kolar'], dtype=object)

In [ ]:
listofdistrict = filtered_works['district_name'].unique()

In [ ]:
listofdistrict

array(['chikkaballapura', 'bengaluru rural', 'kolar'], dtype=object)

In [ ]:
len(listofblocks)

3

In [ ]:
# listofgps = [gp.capitalize() for gp in listofgps]

In [ ]:
# print ("List in proper method", '[%s]' % ', '.join(map(str, listofgps)))

In [ ]:
len(listofgps)

238

In [ ]:
def extractdates(gp):
  gp_lower = gp.lower()
  assetinfo1 = filtered_works[filtered_works['map_gpname'] == gp_lower] #139 rows
  max1 = assetinfo1['work_physically/_completed_date'].max()
  maxdate1 = max1.strftime('%Y-%m-%d')
  maxyr1 = str(max1.year)
  min1 = assetinfo1['work_started_date'].min()
  mindate1 = min1.strftime('%Y-%m-%d')
  minyr1 = str(min1.year)
  month1 = str(min1.month)
  # print(month1)
  # print(minyr1, maxyr1)
  arrmin = mindate1.split('-')
  mindate1 = arrmin[0] + '-' + arrmin[1] + '-01'
  arrmax = maxdate1.split('-')
  maxdate1 = arrmax[0] + '-' + arrmax[1] + '-01'
  # print(mindate1, maxdate1)#dates of availability of assets

  return [minyr1, maxyr1, mindate1, maxdate1]

In [ ]:
  ndvi = pd.read_csv('/content/drive/MyDrive/NREGAproject/daily-ndvi.csv')
  # rainfall = pd.read_csv('/content/drive/MyDrive/NREGAproject/cds-precipitation-radiation.csv')
  # rainfall = rainfall.drop(['surface_net_solar_radiation', '.geo', 'system:index'], axis=1)
  ndvi['date'] = pd.to_datetime(ndvi['date'])

In [ ]:
def preprocessing(gp):
  # ndvi = pd.read_csv('/content/drive/MyDrive/NREGAproject/daily-ndvi.csv')
  # rainfall = pd.read_csv('/content/drive/MyDrive/NREGAproject/cds-precipitation-radiation.csv')
  # rainfall.drop(['surface_net_solar_radiation', '.geo', 'system:index'], axis=1)
  global ndvi
  # rainfall1 = rainfall.pivot_table(values='total_precipitation', columns='gp', index='date')
  # rainfall1.reset_index(inplace= True)
  # rainfall1 = rainfall1.rename_axis(None, axis=1)
  # rainfall1['date'] = pd.to_datetime(rainfall1['date'], format="%Y-%m-%d").dt.date

  if gp in ndvi.columns:
    # gp = listofgps[0]
    gp = gp.capitalize()
    ndvi_abludu = ndvi.loc[:, ['date', gp]]
    ndvi_abludu.rename(columns = {gp :'daily_NDVI'}, inplace = True)
    mylist = extractdates(gp)

    ndvi_abludu['date'] = pd.to_datetime(ndvi_abludu['date'])
    gpndvi = ndvi_abludu.resample('1M', on='date').mean()
    gpndvi['date'] = pd.to_datetime(gpndvi.index)
    gpndvi.set_index('date', inplace=True)
    gpndvi.index = pd.DatetimeIndex(gpndvi.index.values, freq=gpndvi.index.inferred_freq)
    ndvi_before =gpndvi[gpndvi.index.year < int(mylist[0])]
    ndvi_after = gpndvi[gpndvi.index.year >= int(mylist[0])]
    return [ndvi_before, ndvi_after]

In [ ]:
# gp = listofgps[0]
# gp = gp.capitalize()
# ndvi_abludu = ndvi.loc[:, ['date', gp]]
# ndvi_abludu.rename(columns = {gp :'daily_NDVI'}, inplace = True)
# mylist = extractdates(gp)

# ndvi_abludu['date'] = pd.to_datetime(ndvi_abludu['date'])
# gpndvi = ndvi_abludu.resample('1M', on='date').mean()
# gpndvi['date'] = pd.to_datetime(gpndvi.index)
# gpndvi.set_index('date', inplace=True)
# gpndvi.index = pd.DatetimeIndex(gpndvi.index.values, freq=gpndvi.index.inferred_freq)
# ndvi_before =gpndvi[gpndvi.index.year < int(mylist[0])]
# ndvi_after = gpndvi[gpndvi.index.year >= int(mylist[0])]
# ndvi_before

In [ ]:
def senslopeestimator(before, after):
  results = mk.seasonal_test(before)
  results2 = mk.seasonal_test(after)
  before_first = before.iloc[0]['daily_NDVI']
  after_first = after.iloc[0]['daily_NDVI']
  return [results.s, results.z, results.slope, results2.s, results2.z, results2.slope]

In [ ]:
def categorize_trend_before(row):
    S_before = row['slope before']
    Z_before = row['z before']

    # if S_before >= 0.0005 and abs(Z_before) >= 1.96:
    #     return 'significant increase'
    # elif S_before >= 0.0005 and abs(Z_before) < 1.96:
    #     return 'nonsignificant increase'
    # elif S_before < 0.0005 and abs(Z_before) < 1.96:
    #     return 'basically stable'
    # elif S_before < -0.0005 and abs(Z_before) < 1.96:
    #     return 'nonsignificant decrease'
    # elif S_before < -0.0005 and abs(Z_before) >= 1.96:
    #     return 'significant decrease'
    # else:
    #     return 'Undefined'

    if S_before >= 0.0005 and abs(Z_before) >= 1.96:
        return 5
    elif S_before >= 0.0005 and abs(Z_before) < 1.96:
        return 4
    elif abs(S_before) < 0.0005 and abs(Z_before) < 1.96:
        return 3
    elif S_before < -0.0005 and abs(Z_before) < 1.96:
        return 2
    elif S_before < -0.0005 and abs(Z_before) >= 1.96:
        return 1
    else:
        return 0

In [ ]:
def categorize_trend_after(row):
    S_after = row['slope after']
    Z_after = row['z after']

    # if S_after >= 0.0005 and abs(Z_after) >= 1.96:
    #     return 'significant increase'
    # elif S_after >= 0.0005 and abs(Z_after) < 1.96:
    #     return 'nonsignificant increase'
    # elif S_after < 0.0005 and abs(Z_after) < 1.96:
    #     return 'basically stable'
    # elif S_after < -0.0005 and abs(Z_after) < 1.96:
    #     return 'nonsignificant decrease'
    # elif S_after < -0.0005 and abs(Z_after) >= 1.96:
    #     return 'significant decrease'
    # else:
    #     return 'Undefined'
    if S_after >= 0.0005 and abs(Z_after) >= 1.96:
        return 5
    elif S_after >= 0.0005 and abs(Z_after) < 1.96:
        return 4
    elif S_after < 0.0005 and abs(Z_after) < 1.96:
        return 3
    elif S_after < -0.0005 and abs(Z_after) < 1.96:
        return 2
    elif S_after < -0.0005 and abs(Z_after) >= 1.96:
        return 1
    else:
        return 0

In [ ]:
df = pd.DataFrame(columns=['gpname','s before','z before', 'slope before', 's after', 'z after', 'slope after'])
# df2 = pd.DataFrame(columns=['before', 'after'])
for gp in listofgps:
  if isinstance(gp, str):
    gp = gp.capitalize()
    mydatasets = preprocessing(gp)
    if mydatasets is None:
      continue
    else:
      myvals = senslopeestimator(mydatasets[0], mydatasets[1])
      df.loc[len(df.index)] = [gp, myvals[0], myvals[1], myvals[2], myvals[3], myvals[4], myvals[5]]

# Apply the categorization function to create a new column
df['Trend Level Before'] = df.apply(categorize_trend_before, axis=1)
df['Trend Level After'] = df.apply(categorize_trend_after, axis=1)

In [ ]:
pd.set_option('display.max_rows', None)  # Set 'None' to display all rows

In [ ]:
df.to_csv('/content/drive/MyDrive/NREGAproject/senslopemodularized.csv', index=False)

In [ ]:
display(df)

,gpname,s before,z before,slope before,s after,z after,slope after,Trend Level Before,Trend Level After
0,Abludu,-3.0,-0.072405,-0.000374,30.0,0.434631,0.000483,3,3
1,Handiganala,1.0,0.000000,0.000320,-38.0,-0.554529,-0.000343,3,3
2,Anur,16.0,0.457000,0.001164,-24.0,-0.382590,-0.000244,4,3
3,Baktharahalli,16.0,0.457000,0.000628,-68.0,-1.114502,-0.001017,4,3
4,Bashetnahalli,21.0,0.724049,0.000699,-58.0,-0.854274,-0.000578,4,3
5,Thimmanayakanahalli,1.0,0.000000,0.000033,12.0,0.164860,0.000160,3,3
6,Chimangala,-14.0,-0.396067,-0.001082,-102.0,-1.680070,-0.001017,2,3
7,Devaramaulur,14.0,0.396067,0.000865,74.0,1.214308,0.001030,4,4
8,Dibburahalli,20.0,0.578867,0.001348,52.0,0.848352,0.000618,4,4
9,Talakayalabetta,7.0,0.217215,0.000587,52.0,0.764351,0.000633,4,4


In [ ]:
df[df['slope before'] < df['slope after']].count()

gpname                124
s before              124
z before              124
slope before          124
s after               124
z after               124
slope after           124
Trend Level Before    124
Trend Level After     124
dtype: int64

In [ ]:
df[df['slope before'] == df['slope after']].count()

gpname                0
s before              0
z before              0
slope before          0
s after               0
z after               0
slope after           0
Trend Level Before    0
Trend Level After     0
dtype: int64

In [ ]:
df[df['slope before'] > df['slope after']].count()

gpname                101
s before              101
z before              101
slope before          101
s after               101
z after               101
slope after           101
Trend Level Before    101
Trend Level After     101
dtype: int64

In [ ]:
df[df['Trend Level Before'] < df['Trend Level After']].count()

gpname                105
s before              105
z before              105
slope before          105
s after               105
z after               105
slope after           105
Trend Level Before    105
Trend Level After     105
dtype: int64

In [ ]:
df[df['Trend Level Before'] == df['Trend Level After']].count()

gpname                56
s before              56
z before              56
slope before          56
s after               56
z after               56
slope after           56
Trend Level Before    56
Trend Level After     56
dtype: int64

In [ ]:
df[df['Trend Level Before'] > df['Trend Level After']].count()

gpname                64
s before              64
z before              64
slope before          64
s after               64
z after               64
slope after           64
Trend Level Before    64
Trend Level After     64
dtype: int64

To calculate the percentage increase, you can use the formula:

Percentage Increase = (Annual Increase / Initial Value) * 100

The "Annual Increase" is the rate of change per year estimated using the Theil-Sen slope method, in your case, ex: 0.0034.

The "Initial Value" can be taken as the NDVI value in the beginning of the study period.